# 50.039 Theory and Practice of Deep Learning Project 2024

Group 10
- Issac Jose Ignatius (1004999)
- Mahima Sharma (1006106)
- Dian Maisara (1006377)


### Import all relevant libraries

In [24]:
# Numpy
import numpy as np
# Pandas
import pandas as pd

# Torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
print(torchvision.__version__)
import torchvision.transforms as T
from torchvision.transforms import v2
import torchvision.transforms.functional as fn
from torchvision.io import read_image, ImageReadMode
from torchmetrics.classification import MulticlassAccuracy
import math

# File Operations
import os

# import sys
# sys.path.insert(0, '../src')
# from saver_loader import *
# %reload_ext autoreload
# %autoreload 2

0.17.2+cu121


In [25]:
# Use GPU if available, else use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu") 
print(device)

cuda


In [26]:
data_path = os.path.join(os.path.abspath(''), "../../original")
print(data_path)

c:\Users\dianm\Downloads\Dataset\2024_TPDL\notebooks\../../original


### Load in Labels 

In [27]:
# Bug in Path present in training dataset
def fix_error_paths(row):
    row = row.replace("//", "/")
    return row

def str_to_array(row):
    ndarray = np.fromstring(
                row.replace('\n','')
                    .replace('[','')
                    .replace(']','')
                    .replace('  ',' '), 
                    sep=' ')
    return ndarray

In [28]:
labels = ['Pleural Effusion', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity', 'Lung Lesion', 'Consolidation', 'Pneumonia', 'Atelectasis', 'Pneumothorax', 'Edema', 'Pleural Other', 'Fracture', 'Support Devices']

test_df = pd.read_csv("../data/processed/test_one_hot_encoded.csv", index_col=False)
test_df["Path"] = test_df["Path"].apply(fix_error_paths)

for label in labels:
    test_df[label] = test_df[label].apply(str_to_array)

display(test_df)

,Path,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,test/patient64741/study1/view1_frontal.jpg,"[0.0, 0.0, 1.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 1.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0]"
1,test/patient64742/study1/view1_frontal.jpg,"[0.0, 0.0, 1.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]"
2,test/patient64743/study1/view1_frontal.jpg,"[0.0, 0.0, 1.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 1.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]"
3,test/patient64744/study1/view1_frontal.jpg,"[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0]"
4,test/patient64744/study1/view2_lateral.jpg,"[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663,test/patient65236/study1/view1_frontal.jpg,"[0.0, 0.0, 1.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 1.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0]"
664,test/patient65237/study1/view1_frontal.jpg,"[0.0, 0.0, 1.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]"
665,test/patient65238/study1/view1_frontal.jpg,"[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0]"
666,test/patient65239/study1/view1_frontal.jpg,"[0.0, 0.0, 1.0]","[0.0, 0.0, 1.0]","[0.0, 0.0, 1.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 1.0, 0.0]","[0.0, 0.0, 1.0]"


### Custom Dataset implementation

In [29]:
# Implementation of Custom Dataset Class for CheXPhoto Dataset
class CheXDataset(Dataset):
    # Accepts dataframe object and str
    def __init__(self, df: pd.DataFrame, px_size: int = 256):
        self.dataframe = df.copy()
        self.px_size = px_size
        self.transform = T.Compose([
            v2.Resize((self.px_size, self.px_size), interpolation=T.InterpolationMode.BICUBIC)
        ])

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        x_path = data_path + "/" + self.dataframe.iloc[idx, 0].split("CheXphoto-v1.0", 1)[-1]
        resized_x_tensor = self.transform(read_image(x_path, mode = ImageReadMode.RGB)) /255
        y = torch.tensor(self.dataframe.iloc[idx, 1]).type(torch.LongTensor)
        return resized_x_tensor, y

In [30]:
def preprocess_pEffClassification(df):
    df.drop(columns=['Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity', 'Lung Lesion', 'Consolidation', 'Pneumonia', 'Atelectasis', 'Pneumothorax', 'Edema', 'Pleural Other', 'Fracture', 'Support Devices'], axis=1, inplace=True)
    return df

def preprocess_cardioClassification(df):
    df.drop(columns=['Enlarged Cardiomediastinum', 'Pleural Effusion', 'Lung Opacity', 'Lung Lesion', 'Consolidation', 'Pneumonia', 'Atelectasis', 'Pneumothorax', 'Edema', 'Pleural Other', 'Fracture', 'Support Devices'], axis=1, inplace=True)
    return df

def ohe_to_class(row):
    if np.sum(row) > 0:
        return np.argmax(row)
    else:
        return -100

# Creates copy of train_df for pEff and cardio
test_p = test_df.copy()
test_c = test_df.copy()

In [31]:
# Drop all other labels
pEff_test = preprocess_pEffClassification(test_p)
# Replace OHE with int class labels
pEff_test["Pleural Effusion"] = pEff_test["Pleural Effusion"].apply(ohe_to_class)
display(pEff_test)

,Path,Pleural Effusion
0,test/patient64741/study1/view1_frontal.jpg,1
1,test/patient64742/study1/view1_frontal.jpg,1
2,test/patient64743/study1/view1_frontal.jpg,1
3,test/patient64744/study1/view1_frontal.jpg,1
4,test/patient64744/study1/view2_lateral.jpg,1
...,...,...
663,test/patient65236/study1/view1_frontal.jpg,1
664,test/patient65237/study1/view1_frontal.jpg,1
665,test/patient65238/study1/view1_frontal.jpg,1
666,test/patient65239/study1/view1_frontal.jpg,1


In [32]:
# Drop all other labels
cardio_test = preprocess_cardioClassification(test_c)
# Replace OHE with int class labels
cardio_test["Cardiomegaly"] = cardio_test["Cardiomegaly"].apply(ohe_to_class)
display(cardio_test)

,Path,Cardiomegaly
0,test/patient64741/study1/view1_frontal.jpg,2
1,test/patient64742/study1/view1_frontal.jpg,1
2,test/patient64743/study1/view1_frontal.jpg,2
3,test/patient64744/study1/view1_frontal.jpg,1
4,test/patient64744/study1/view2_lateral.jpg,1
...,...,...
663,test/patient65236/study1/view1_frontal.jpg,2
664,test/patient65237/study1/view1_frontal.jpg,1
665,test/patient65238/study1/view1_frontal.jpg,1
666,test/patient65239/study1/view1_frontal.jpg,2


### Custom Dataloader

In [33]:
# Load into custom Dataset
pEff_test_data = CheXDataset(pEff_test)
cardio_test_data = CheXDataset(cardio_test)

# Load into DataLoader
batch_size = 128
pEff_test_loader = DataLoader(pEff_test_data, batch_size)
cardio_test_loader = DataLoader(cardio_test_data, batch_size)

In [34]:
x, y = pEff_test_data[0]
print(x, x.shape, x.dtype)
print(y, y.shape, y.dtype)

tensor([[[0.0275, 0.0314, 0.0235,  ..., 0.0314, 0.0314, 0.0314],
         [0.0157, 0.0235, 0.0118,  ..., 0.0314, 0.0314, 0.0314],
         [0.0118, 0.0235, 0.0235,  ..., 0.0196, 0.0314, 0.0314],
         ...,
         [0.0824, 0.1176, 0.1961,  ..., 0.2314, 0.1569, 0.2588],
         [0.0824, 0.1294, 0.2078,  ..., 0.2314, 0.1686, 0.2784],
         [0.0824, 0.1451, 0.2196,  ..., 0.2392, 0.1725, 0.2471]],

        [[0.0275, 0.0314, 0.0235,  ..., 0.0314, 0.0314, 0.0314],
         [0.0157, 0.0235, 0.0118,  ..., 0.0314, 0.0314, 0.0314],
         [0.0118, 0.0235, 0.0235,  ..., 0.0196, 0.0314, 0.0314],
         ...,
         [0.0824, 0.1176, 0.1961,  ..., 0.2314, 0.1569, 0.2588],
         [0.0824, 0.1294, 0.2078,  ..., 0.2314, 0.1686, 0.2784],
         [0.0824, 0.1451, 0.2196,  ..., 0.2392, 0.1725, 0.2471]],

        [[0.0275, 0.0314, 0.0235,  ..., 0.0314, 0.0314, 0.0314],
         [0.0157, 0.0235, 0.0118,  ..., 0.0314, 0.0314, 0.0314],
         [0.0118, 0.0235, 0.0235,  ..., 0.0196, 0.0314, 0.

In [35]:
x, y = cardio_test_data[0]
print(x, x.shape, x.dtype)
print(y, y.shape, y.dtype)

tensor([[[0.0275, 0.0314, 0.0235,  ..., 0.0314, 0.0314, 0.0314],
         [0.0157, 0.0235, 0.0118,  ..., 0.0314, 0.0314, 0.0314],
         [0.0118, 0.0235, 0.0235,  ..., 0.0196, 0.0314, 0.0314],
         ...,
         [0.0824, 0.1176, 0.1961,  ..., 0.2314, 0.1569, 0.2588],
         [0.0824, 0.1294, 0.2078,  ..., 0.2314, 0.1686, 0.2784],
         [0.0824, 0.1451, 0.2196,  ..., 0.2392, 0.1725, 0.2471]],

        [[0.0275, 0.0314, 0.0235,  ..., 0.0314, 0.0314, 0.0314],
         [0.0157, 0.0235, 0.0118,  ..., 0.0314, 0.0314, 0.0314],
         [0.0118, 0.0235, 0.0235,  ..., 0.0196, 0.0314, 0.0314],
         ...,
         [0.0824, 0.1176, 0.1961,  ..., 0.2314, 0.1569, 0.2588],
         [0.0824, 0.1294, 0.2078,  ..., 0.2314, 0.1686, 0.2784],
         [0.0824, 0.1451, 0.2196,  ..., 0.2392, 0.1725, 0.2471]],

        [[0.0275, 0.0314, 0.0235,  ..., 0.0314, 0.0314, 0.0314],
         [0.0157, 0.0235, 0.0118,  ..., 0.0314, 0.0314, 0.0314],
         [0.0118, 0.0235, 0.0235,  ..., 0.0196, 0.0314, 0.

### Second iteration - Basic Convolutional neural network (CNN)

In [36]:
class ConvolutionBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, kernel_size: int, stride: int, padding: int, pool_size: int = 2):
        super(ConvolutionBlock, self).__init__()
        
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.bn = nn.BatchNorm2d(out_channels)
        self.activation = nn.ReLU()
        self.pooling = nn.MaxPool2d(pool_size, pool_size)
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.activation(out)
        out = self.pooling(out)
        return out

In [37]:
class FCBlock(nn.Module):
    def __init__(self, in_features: int, out_features: int, dropout_rate: int=0.2):
        super(FCBlock, self).__init__()
        
        self.fc = nn.Linear(in_features, out_features, dtype=torch.float32)
        self.activation = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
    
    def forward(self, x):
        out = self.fc(x)
        out = self.activation(out)
        out = self.dropout(out)
        return out

In [38]:
class ClassifierBlock(nn.Module):
    def __init__(self, in_features: int, hidden_features: tuple[int], out_features: int, dropout_rate: int = 0.2):
        super(ClassifierBlock, self).__init__()

        self.inputs = FCBlock(in_features, hidden_features[0], dropout_rate)
        self.layers = nn.Sequential(*[FCBlock(hidden_features[i], hidden_features[i+1], dropout_rate) for i in range(len(hidden_features)-1)])
        self.linear = nn.Linear(hidden_features[-1], out_features, dtype=torch.float32)
    
    def forward(self, x):
        out = self.inputs(x)
        out = self.layers(out)
        out = self.linear(out)
        return out

In [39]:
class ConvolutionalNN(nn.Module):
    def __init__(self, channels: tuple[int], img_size: tuple[int], hidden_size: tuple[int], output_size: int, dropout_rate: int = 0.2):
        super(ConvolutionalNN, self).__init__()
        
        # Hyperparameters for Convolutional Layers
        self.kernel_size = 3
        self.stride = 1
        self.padding = 1
        self.pool_size = 2

        # Input size of Image
        self.input_height, self.input_width = img_size

        # Calculate Input Size of Classifier
        for i in range(len(channels) - 1):
            self.input_height = math.floor((self.input_height + 2 * self.padding - self.kernel_size)/ self.stride + 1) // self.pool_size
            self.input_width =  math.floor((self.input_width  + 2 * self.padding - self.kernel_size)/ self.stride + 1) // self.pool_size
        
        self.classifier_size = channels[-1] * self.input_height * self.input_width
        
        # Model Layers
        self.conv_layers = nn.Sequential(*[ConvolutionBlock(channels[i], channels[i+1], self.kernel_size, self.stride, self.padding, self.pool_size) for i in range(len(channels)-1)])
        self.classifier1 = ClassifierBlock(self.classifier_size, hidden_size, output_size, dropout_rate)

    def forward(self,x):
        # Convolutional Layers
        out1 = self.conv_layers(x)

        # keep batch size and flatten the rest
        out2 = out1.view(out1.size(0), -1)

        # Classifier Layers
        out3 = self.classifier1(out2)
        
        return out3

In [40]:
from tqdm.notebook import tnrange, tqdm
from torchmetrics.classification import MulticlassAccuracy

def test_loop(model, test_loader, loss):
    model.eval()

    test_loss = 0
    test_total = 0
    test_accuracy = 0

    accuracy = MulticlassAccuracy(num_classes=3, ignore_index=-100).to(device)

    with torch.no_grad():
        for inputs, outputs in tqdm(test_loader):
            inputs_re, outputs_re = inputs.to(device), outputs.to(device)
            preds = model(inputs_re)

            # Compute loss
            loss_value = loss(preds, outputs_re)

            # Compute metrics
            test_loss += loss_value.item() * outputs_re.size(0)
            test_total += outputs_re.size(0)

            accuracy.update(preds, outputs_re)

    test_loss /= test_total
    test_accuracy = accuracy.compute()

    print(f'--- Val Loss: {test_loss:.4f}, Train accuracy: {test_accuracy:.4f}\n')

    return test_loss, test_accuracy

In [41]:
def test(model, weights, test_loader):
    # loss function
    loss = nn.CrossEntropyLoss(weight=torch.tensor(weights), ignore_index=-100).to(device)
        
    # Test loop
    test_loss, test_accuracy = test_loop(model, test_loader, loss)

    print(f"Test loss: {test_loss:.4f}, Test accuracy: {test_accuracy:.4f}")

In [42]:
# Simplistic one for now, if need the full checkpoint, refer to helpers.py
def restore_model(model, path='../bin'):
    model_path = os.path.join(path, '{}-.ckpt')
    model.load_state_dict(torch.load(model_path)['model_state_dict'])

In [43]:
# Create Model
channels = (3, 8, 16, 32, 128)
input_size = (256, 256)
hidden_size = (8192, 512) 
output_size = 3
dropout_rate = 0.2

model_pEff = ConvolutionalNN(channels, input_size, hidden_size, output_size, dropout_rate).to(device)
restore_model(model_pEff, "./bin/pEff-.ckpt")
print(model_pEff)

TypeError: restore_model() missing 1 required positional argument: 'path'

In [ ]:
pEff_weights = [1873/19582, 4976/19582, 12733/19582]
test(model_pEff)

  0%|          | 0/6 [00:00<?, ?it/s]

--- Val Loss: 1.1087, Train accuracy: 0.5000

Test loss: 1.1087, Test accuracy: 0.5000


In [ ]:
# Create Model
channels = (3, 8, 16, 32, 128)
input_size = (256, 256)
hidden_size = (8192, 512) 
output_size = 3
dropout_rate = 0.2

model_cardio = ConvolutionalNN(channels, input_size, hidden_size, output_size, dropout_rate).to(device)
restore_model(model_cardio, "./bin/cardio-.ckpt")
print(model_cardio)

ConvolutionalNN(
  (conv_layers): Sequential(
    (0): ConvolutionBlock(
      (conv): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ReLU()
      (pooling): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): ConvolutionBlock(
      (conv): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ReLU()
      (pooling): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (2): ConvolutionBlock(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ReLU()
      (pooling): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode

In [ ]:
cardio_weights = [1149/6812, 1623/6812, 4040/6812]
test(model_cardio, cardio_weights, cardio_test_loader)

  0%|          | 0/6 [00:00<?, ?it/s]

--- Val Loss: 1.0950, Train accuracy: 0.5000

Test loss: 1.0950, Test accuracy: 0.5000
